In [1]:
from langchain.llms import LlamaCpp, OpenAI
import config
import os

In [2]:
API = os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

# Cadenas
Las cadenas en LangChain funcionan para poder crear flujos de trabajo, donde unimos distintos "bloques" para poder crear un sistemas con LLMs mas complejo.

Por ejemplo, si quieres crear un sistema el cual conecte distintas entradas y salidas de LLMs entre si con las cadenas puedes gestionar que modelo genera que información, con que prompt y la salida de ese modelo si funciona como entrada a otro.

Hay bastantes cadenas que [LangChain tiene por default](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.chains) que puedes usar como bloques para crear tu sistema. Pero tambien puedes crear tus propias cadenas al igual que descargar distintas cadenas que ha hecho la comunidad desde el [LangChain-Hub](https://github.com/hwchase17/langchain-hub)

## Cadenas más usadas
Hay bastantes cadenas las cuales estan integradas dentro de LangChain, pero en este ejemplo estaremos viendo las mas cómunes que y muy útiles para poder desarrollar distintos sistemas.

* LLMChain
* SequentialChain
* Math/transformation

### LLMChain
Lo que hace es unir dos elementos para que puedas interactuar con las LLMs de manera mas sencilla.

Una un modelo LLM (Puede ser LLama, OpenAI, Cohere etc.) y los templates de prompts vistos en el cuaderno [Langchain 1.ipynb](./Langchain%201%20-%20Modelos%20y%20Prompts.ipynb).

In [3]:
from langchain import LLMChain, OpenAI, PromptTemplate

In [4]:
prompt = '''Eres un asistente virtual experto en {tema} y respondes
            con una lista de 3 conceptos clave sobre el mismo
            Solo enumeras los tres conceptos'''
template = PromptTemplate.from_template(prompt)

In [5]:
llm = OpenAI(openai_api_key=API)
cadena_LLM = LLMChain(llm=llm, prompt=template)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


La cadena es el elemento con el que vas a intecartuar ahora para poder hacer predicciones.

Lo que hace ahora esta cadena es tomar tu input (El tema), darle formato al prompt que usará y envia el prompt construido al modelo para su generación de texto

In [6]:
cadena_LLM.predict(tema="ingenieria civil")

' clave \n\n1. Diseño estructural: Es la disciplina que se encarga de crear la estructura de los edificios y otras construcciones, teniendo en cuenta la resistencia de los materiales y la estabilidad de la estructura.\n\n2. Mecánica de suelos: Se refiere al estudio de las propiedades físicas y mecánicas del suelo y su relación con las construcciones que se van a realizar sobre él.\n\n3. Normativas y códigos de construcción: Son los estándares y regulaciones que deben seguirse en la elaboración de proyectos de ingeniería civil para garantizar la seguridad y calidad de las construcciones.'

### SequentialChain
Para muchos casos de usos solo enviar un texto para ser procesado no es suficiente, por lo que se requiere de una secuencia de procesos que se ejecuten en orden. Para esto se puede utilizar las cadenas `SimpleSequentialChain o SequentialChain` que permiten encadenar varios procesos de manera secuencial.

Cuando quieres que la salida que genera el modelo funcione como entrada para otro, este es una gran manera de hacerlo. 

En este ejemplo veremos SequentialChain, ya que nos brinda mas flexibilidad que SimpleSequentialChain, pues puede recibir multiples entradas y generar multiples salidas.

Armamos la primera cadena que será la misma que hicimos en el ejercicio anterior

In [7]:
llm = OpenAI(openai_api_key=API)
prompt = '''Eres un asistente virtual experto en {tema} y respondes
            con una lista de 3 conceptos clave sobre el mismo
            Solo enumeras los tres conceptos'''
template = PromptTemplate.from_template(prompt)
cadena_lista = LLMChain(llm=llm, prompt=template, output_key="lista_conceptos")

Armamos una cadena la cual va a recibir la salida de la cadena cadena_LLM y lo procesa para generar otro texto

In [8]:
prompt = '''Eres un asistente virtual que recibe una lista de conceptos
            de un area de conocimiento y
            debe devolver cual de esos conceptos es mejor aprender primero.
            Los conceptos son: {lista_conceptos}'''
template = PromptTemplate.from_template(prompt)
cadena_inicio = LLMChain(llm=llm, prompt=template, output_key="donde_iniciar")

In [9]:
from langchain.chains import SequentialChain
cadenas = SequentialChain(chains=[cadena_lista, cadena_inicio],
                          input_variables=["tema"],
                          output_variables=["lista_conceptos", "donde_iniciar"],
                          verbose=True)

In [10]:
cadenas({"tema": "programacion"})

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'tema': 'programacion',
 'lista_conceptos': '\n\n1. Lenguaje de programación: Es un conjunto de reglas y símbolos que permiten escribir instrucciones para que una computadora realice una tarea específica.\n\n2. Algoritmo: Es un conjunto ordenado de pasos que se deben seguir para resolver un problema o realizar una tarea en un programa de computadora.\n\n3. Variables: Son espacios de memoria utilizados para almacenar y manipular datos en un programa de computadora, permitiendo que estos datos puedan ser modificados durante la ejecución del programa. ',
 'donde_iniciar': '\n\n4. Control de flujo: Es la forma en la que un programa de computadora controla la ejecución de sus instrucciones, permitiendo que estas se ejecuten en un orden específico o que se repitan un número determinado de veces.\n\n5. Estructuras de datos: Son formas de organizar y almacenar datos en una computadora, permitiendo que estos puedan ser accedidos y modificados de manera eficiente.\n\n6. Funciones: Son bloques d

In [11]:
from langchain.chains import SimpleSequentialChain
cadena_simple = SimpleSequentialChain(chains=[cadena_lista, cadena_inicio], verbose=True)
cadena_simple.run("Inteligencia artificial")



> Entering new SimpleSequentialChain chain...


C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 sin añadir ningún comentario

1. Aprendizaje automático
2. Redes neuronales
3. Procesamiento del lenguaje natural


El mejor concepto para aprender primero sería Aprendizaje automático.

> Finished chain.


'\n\nEl mejor concepto para aprender primero sería Aprendizaje automático.'

## Otros ejemplos

In [12]:
# %pip install numexpr

In [13]:
# MathChain
from langchain import LLMMathChain
cadena_mate = LLMMathChain(llm=llm, verbose=True)
cadena_mate.run("Cuanto es 432*12-32+32?")



> Entering new LLMMathChain chain...
Cuanto es 432*12-32+32?

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


```text
432*12-32+32
```
...numexpr.evaluate("432*12-32+32")...

Answer: 5184
> Finished chain.


'Answer: 5184'

Cuando vamos a hacer un preprocesamiento del promp

In [14]:
# TransformChain
from langchain.chains import TransformChain

def eliminar_brincos(input):
    """Elimina los brincos de línea de un texto."""
    texto = input["texto"]
    return {"texto_limpio": texto.replace("\n", " ")}


cadena_transformacion = TransformChain(input_variables=["texto"],
                                        output_variables=["texto_limpio"],
                                        transform=eliminar_brincos)

prompt = '''\nEste es un texto \ncon brincos de\n línea.\n\n'''


cadena_transformacion.run(prompt)


' Este es un texto  con brincos de  línea.  '